# Computing transitive closure on a relations and graphs

Review slides https://pages.mtu.edu/~nilufer/classes/cs2311/2012-march/cs2311-s12-ch9-relations-part2.pdf#slide.1
This clarifies that transitive closure is something we can compute by thinking of a relation as a graph.
It also clarifies that the complete transitive closure is the power of the matrix raised to n (number of elements) because that covers all paths up to length n.

A relation can be visualized as a bipartite (two part) graph. 
In [[https://rpubs.com/pjmurphy/317838][R we can visualize a bipartite graph]] directly with plot function.
This builds on [[https://www.rdocumentation.org/packages/igraph/versions/1.2.4.2/topics/graph_from_adjacency_matrix][creating a graph from the adjacency matrix.]]
When we have two distinct sets, we need to construct the adjacency matrix that has vertices combined from all the elements of both sets.
It's easy to [[https://en.wikipedia.org/wiki/Adjacency_matrix#Of_a_bipartite_graph][construct this adjacency matrix of a bipartite graph]].

We can draw these networks in R.
[[https://hal.archives-ouvertes.fr/hal-01722543/][It might be easier to use the ggplot2 routines]].  https://hal.archives-ouvertes.fr/hal-01722543/
Or, once defined in a standard format, a dedicated tool for network visualization.
[[https://medium.com/@Elise_Deux/list-of-free-graph-visualization-applications-9c4ff5c1b3cd][There are many to choose from.]]

We learn by discovering relationships between ideas.

We can document these connections using the notation of sets and recording the relations between items in those sets.
Combining these relations can help us discover new connections between items. Combining relations is call relation composition.

We can "see" relations by viewing them as a graph.
In graph form, relations are treated as networks of connected nodes.
The connections between items in the relation are visualized as edges between the nodes in the graph.
Graphs allow us to visually explore connected items and discover new relationships by following the edges that make up the paths between nodes.

For example, social networks contain many familiar relations like "friend of" and can be drawn as a graph where nodes represent people and edges between them represent their friendships.
Another network we interact with every day is the World Wide Web.   Relations between web pages are defined by the hyperlinks between pages. The web can be visualized as a network called the web graph.  Here the web pages are nodes and the links between web pages are the edges.

We might also have data sets that describe two different relations between items and composing them could help discover secondary relationships.  For example, there are data sets that document drugs which are known to activate specific genes and other data sets that tells us which genes activate other genes. By composing these relations, we can discover drugs which could be investigated to activate genes indirectly through other genes.  This would give us a way to control genes that can't be directly controlled through known drug interactions. 

Let's summarize these ideas with an example of a relation draw as a graph. It is a simple social network for the "friends of" relation between four imaginary people $\{ a, b, c, d \}$. We draw it using the [R's igraph library](https://igraph.org/r/doc/) and powerful plotting routines.  The nodes in the graph represent the people and the edges their "friends of" relation.  We can visually compose the "friend of a friend" transitive relation and discover that $(b,d)$ are members of that relation because $b$ is a "friend of" $c$ and $c$ is a "friend of" $d$.

In [ ]:
library(igraph)
options(repr.plot.width=5, repr.plot.height=5)
friends <- matrix(c(0,0,1,0, 0,0,1,0, 1,1,0,1,  0,0,1,0), nc=4,
               dimnames=list(c('a', 'b', 'c', 'd'), c('a', 'b', 'c', 'd')))
g <- graph_from_adjacency_matrix( friends, diag=FALSE, mode="undirected" )
plot(g)

## Sets and relations

Let's review the language of relations and their composition.

Let's say we have a set of people $\{ a, b, c, d \}$ and know a parent-child relation, $R = \{(a,b)\}$,  and two sibling relations, $S=\{(b,c),(b,d)\}$, on that set.
Given this information, we can compose, or compute, the relation of aunts or uncles between items in the set.
That is we can deterime the composed relation $S \circ R=\{(a,c),(a,d)\}$, which tells us that person $a$ is a "nephew or neice of" persons $c$ and $d$.
Or conversly, that persons $c$ and $d$ are an "aunt or uncle of" person $a$.
An aunt (or uncle) is a person who is a sibling of a parent.
Therefore, if we know $a$ is a "child of" $b$, $aRb$, and $b$ is "sibling of" $c$ and $d$,  $bSc$ and $bSd$, then the composition of the relation, $R \circ S$, tells us which people have aunt or uncle relationships.

This new, composed relation $S \circ R$ defines a transitive relation.
Transitive means if $a$ is related to $b$ and $b$ is related to $c$ then $a$ is also related to $c$.
In other words, the connection between $a$ and $c$ transits across $b$.
In graph terms, there is a path between $a$ and $c$ that passes through $b$.

Transitive relationships can transit any number of intermediate items.
For example, we could have a transitive relationship between $a$ and $d$ because we know relationships where $a$ is related to $b$, $b$ is related to $c$, and $c$ is related to $d$. In a graph, we would call that a path between $a$ and $d$ of length three.

Transitivity is a property of some relations. Not all relations are transitive. 
When a relation is transitive, however, it provides a powerful tool for discovering connections between items.

The transitive closure of a set contains all the relationships in a set that result from applying the transitive relation on the set.
A closure is just the complete collection of all the relationships in which we are interested.
The transitive closure can be computed by following all the connections between the nodes that are defined by the transitive relation, repeatedly applying the relation to the set until all pairs are documented.
If we have a relation $R$, we can compute the transitive closure $R^*$, which is a new relation that contains the original relation and the complete collection of transitive relationships of any length between items in the set.

## Matrix notation for relations

Relations can be conveniently represented as a matrix.
The $n$ items in the set define the $n$ rows and $n$ columns of an $nxn$ matrix.
The relation is represented by putting a one in the matrix entry $i,j$ if there is a relation between element $i$ and $j$ of the set and a zero if there is not.
This gives us a binary matrix of $1$'s and $0$'s.

Let's represent our relations above in matrix form.
We'll use the R language in these examples, mainly because it has native support for matrix representations which simplifies this discussion and it has great [plotting support if we want to visualize our networks](https://hal.archives-ouvertes.fr/hal-01722543).

Let's define a list to represent the items in our set $\{a, b, c, d\}$.  These are names of our people.  Defining them as a list makes it easier to use the names when we we create the matrix. Good labels make data easier to read.

In [ ]:
person = c('a','b','c','d')

Our first relation $R$ described the "child of" relation between $a$ and $b$.  Let's represent this in matrix form as $M_R$

[In R we create a matrix with the matrix() function](https://www.tutorialspoint.com/r/r_matrices.htm) and pass it our data, matrix dimensions, and names for the dimensions. We'll use the person names to name the dimensions.  Since we have four items in the set, the matrix will be four rows by four columns and have 16 entries.  The data is represented as a list of zeros and ones, where a one represents a relation between two items in our set.  We will read the entries by row to make the input list a little easier to read for those of us who read matrices in row-major order.  In this example, the second entry in the list will populate the cell in row $a$ and column $b$, meaning $a$ is a "child of" $b$.

In [ ]:
M_R <- matrix( c(0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0), nrow=4, byrow=TRUE, dimnames=list(person,person))
print(M_R)

We'll also create a matrix $M_S$ for our "sibling of" relation.  Recall that $b$ is a sibling of $c$ and $d$, which means there are ones in entries $(b,c)$ and $(b,d)$ of the matrix.

In [ ]:
M_S <- matrix( c(0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0), nrow=4, byrow=TRUE, dimnames=list(person,person))
print(M_S)

## Composing Relations with Matrix Multiplication

Composing relations represented as a matrix is simple.
[We can use matrix multiplication to compute transitive relations.](http://www.facweb.iitkgp.ac.in/~niloy/COURSE/Autumn2008/DiscreetStructure/scribe/Lecture07CS1039.pdf)
If we have two relations $R$ and $S$ over a set of $n$ items, then we an represent each relation in its $nxn$ binary matrix form as $M_R$ and $M_S$.
The composition of relation $S \circ R$ is the multiplication of matrix $M_R$ and matrix $M_S$.  Concisely,  $S \circ R = M_{(R \circ S)} = M_R \odot M_S$.  Binary matrix multiplication, indicated by $\odot$, is discussed below.

This composition gives the transitive relations of length two, i.e. $aRb$ and $bSc$. We can see from the  result of the matrix computation that $a$ is indeed a "neice or nephew of" $c$ and $d$ because the entries $(a,c)$ and $(a,d)$ contain a $1$.

In [ ]:
M_RS=M_R %*% M_S
print(M_RS)

## Transitive Closure and Matrix Multiplication

More generally, we can compute transitive relations of any length by repeated matrix multiplication.
Each multiplication increases the path length by one.

This makes sense.
If we start with the matrix $M_R$, it contains all the relations of length one.
That is, direct relationships between any pair of items in the set.
If $R$ contains transitive relations, then we can resolve any missing pairs.
If we multiply $M_R * M_R$, we get the transitive relations of length two.
If we do it again and multiply $M_R * (M_R * M_R)$ we get the transitive relations of length three.
The longest interesting path is at most n-1 steps away.
This would be a transitive relation composed by stepping through each item in the set.
This means we can compute the transitive closure of a relation $R$ on an $n$-item set by multiplying the $M_R$ matrix by itself $n-1$ times.
This is simply the $n^{th}$ power of the matrix.
Therefore the transitive closure $R^*$ can be computed as $M_{(R^*)} = (M_R)^n$.

Our use of matrix multiplication for relation composition has been ignoring a detail about the summation step in matrix multiplication.
We know that $1+1=2$.
Using matrix multiplication for relation composition, if there are any rows and columns that both have $1$'s in the same dimension, indicating a transitive relation, the summation will be $2$, $3$, or what ever number of connections exist between those two items.
Since we are only interested in the existance of a relation, the only value ever needed in the result is a $1$, indicating the presence of a relation.
We can use Boolean matrix multiplication, represented by $\odot$, to achieve that outcome.
In Boolean math we use logical AND for multiplication and logic OR for addition.
Here, $1 x 1$ is still $1$ and $1 x 0$ and $0 x 0$ are still $0$.
Boolean multiplication retains the relation filtering function.
But now, $1 + 1 = 1$.
Boolean addition records the existance of a relation without being concerned about the degree of connectivity.

We can still substitute ordinary matrix multiplication for Boolean matrix operation. All we do is normalize all non-zero elements in the final result to $1$. This makes sure the final values remain $1$ and $0$.

## The Adjacency Matrix and Graph Connectivity

The relation-as-a-matrix representation makes transitive closure computation extremely useful for determining the connected nodes in a graph.
A graph of $n$ verticies can be represented as an $nxn$ adjacency matrix.
The adjacency matrix has a value of $1$ for entry $i,j$ if the vertices $i$ and $j$ are connected by an edge in the graph and a value of $0$ if there is no edge.

An adjacency matrix is identical to a set representation.
Graphs like social networks or maps of the web can be represented as an adjacency matrix.
Computing the transitive closure of these "relations" identifies all connected nodes in the graph; nodes connected either directly or by a path through intermediate nodes.

It's worth noting that edges, like relations, are not required to be symmetric.
In a directed graph, an edge between $i,j$ doesn't guarantee there is an edge between $j,i$.
A familiar example is the non-symmetric "follows" relation on Instagram.
Just because $a$ follows $b$ doesn't mean $b$ follows $a$, $a=b$ does not mean $b=a$.
A symetric relation describes an undirected graph.
An edge between $i,j$ means there is an edge between $j,i$.
Both entries in the adjaceny matrix are $1$.
For symmetric relations and undirected graphs, the adjacency matrices are square and symmetric.

## Computing Friends of Friends

The "friend" relation can be used to compute a "friend of a friend" relation.
This is the equivalent of finding nodes in the friends social network that are connected by one intermediate node.
In other words, the path length between nodes in the original graph is two.
The composed "friend of a friend" relation is simply the square of the original "friend" relation.
$M_{f \circ f} = (M_{f})^2$

We can easily see friends of friends by plotting $M_{f \circ f}$.
The nodes $a$, $b$, and $d$ are now directly connected in the new relation as friends of friends.
Because $c$ knows all people as friends, it is an unconnected node in the friend of a friend graph.
Composed relations are a powerful tool for communitity detection in graphs.

In [ ]:
fof = friends %*% friends > 0
g <- graph_from_adjacency_matrix(fof , diag=FALSE, mode="undirected" )
plot(g)

## Compute Performance Considerations

Multiplying logical matrices makes easy work of composing new relations.
Visualizing the new ideas represented by composed relations strengthens our understanding.

Whenever we ask a machine to work for us, however, we need to be interested in, or concern ourselves with, the amount of work we are actually requesting it to carry out.
Work on a computer is measured in instructions.
The machine has to execute many instructions to accomplish the tasks we have set forth.

The time it takes to carry out those instructions is governed by two factors.
The raw speed of the computer as measured in cycles per second.
Generally speaking, a computer can execute one instruction per cycle.
Modern processors use pipelines to keep track of many instructions at a time, with the fastest ones able keep 32 or more instructions moving forward at a time.

With clock speeds around 3GHz and deep pipelines, we can expect our fastest processors to carry out as many as 100 billion instructions per second.
The simpler the instructions the better, with addition being as simple as it gets.
Keep in mind that multiplication really represents repeated addition operations.

The second factor affecting performance is how fast data can move through the processor.
For a processor to add billions of numbers in a second, it needs to have billions of numbers arriving each second.
When data isn't available, the processor starves.
The cycles it executes go unused.

Making sure the data arrives in time involves planning.
Most of that is managed by the processor itself.
Fast data caches located directly on the processor hold on to the most frequently used data. 
There it can be moved around at processor speeds.
Multiple cache layers work together to gather data from the relatively slow computer memory.
Planning how computations are organized ensures data efficiently moves through the processor and avoids starvation.

## The Computational Cost of Matrix Multiplication

So, is it practical to simply multiply matrices to compose relations or to compute the transitive closure?
We might get away with a naive approach when data sets are small.
Naive means brute force computation.
Brute force computation carry out all the multiplication and addition instructions expected for standard matrix multiplication.

Multiplying two $nxn$ matrices requires carring out $n^3$ operations; there are $n^2$ entries in an $nxn$ matrix and each entry is involved in $n$ multiply-add steps.
Therefore, the naive algorithm for mulitiplying two matrices requires on the order of $nxnxn$ or $n^3$ operations.
This approach is bounded by an $O(n^3)$ (read, [big-oh](https://en.wikipedia.org/wiki/Big_O_notation) n-cubed) order computation.
Additionally, the transitive closure requires $n-1$ matrix multiplications.
This means a naive transitive closure computation is really bounded by an $O(n^4)$ order computation.

If our data set is large we should avoid naive computation.
Say we have 100 element set.
A naive computation of transitive closure will require on the order of $100^4$ or 100 million operations.
This is probably not too much to ask of a modern computer.
But remember, the work increases exponentially for every factor of 10 increase in our set of elements.
Data becomes big quickly at that rate.

A 1000 element set will take 1000 times longer than the 100 million operations it took for a 100 element set.
This already accounts for all the instructions it is possible to run in one second on our fastest computers.
Increase the data set by another factor of 10, and now we are waiting 15 minutes.
A set with ten thousand items is not uncommon in an era of big data.
At that scale, we can quickly run into significant wait times or, worse, choose to limit the amount of information from which we are willing to learn.

For example, when modeling gene relationships, social networks or the web, we can easily imagine networks with thousands, millions or even billions of nodes.
Even if we are only trying to composed two or three relations, nodes connected by one or two intermediate nodes, we are still bounded by $O(n^3)$ order computations.
Composing a relation with a million items would take a whole season.
Naive computations quickly become unmanagable at this scale.

## Fast Matrix Multiplication

In 1969 Volker Strassen showed that matrix multiplication could be organized in a way that avoids having to carry out the full set of operations required by a naive approach.
This insight introduced the notion of fast matrix multiplication.
[Strassen's algorith](https://en.wikipedia.org/wiki/Strassen_algorithm) demonstrated performance bounded by $O(n^{\log_{2}7}) \approx O(n^{2.8})$.
Fast matrix multiplication methods have been improved since.
The [state of the art algorithms for fast matrix multiplication](https://en.wikipedia.org/wiki/Matrix_multiplication#Computational_complexity) are bounded by $O(n^{2.373})$.
Keep in mind, however, if we want the transitive closure of a set we still need $n - 1$ of these multiplications.
This bumps performance back up to $O(n^{3.373})$, but
we are still better off than the $O(n^4)$ performance of the naive approach.

Are there faster methods?
The Floyd-Warshall algorithm for computing the shortest path between all pairs of nodes in a weighted graph can be applied to compute transitive closure.
The advantages of representing relations in matrix form and extending that representation to graphs through the adjacency matrix should now be clear.
These interchangable representations let us consider solutions from a number of domains.

Floyd-Warshall runs in $O(n^3)$, for $n$-node networks.
We don't really care about knowing weights for transitive closure, nonetheless, knowing the cheapest path through a network is a nice insight to have for many data sets.
Floyd-Warshall achieves its $O(n^3)$ performance using a programming technique called dynamic programming, which breaks computations into overlapping steps.
The algorith inspects paths of increasing length between each node and keeps track of the shortest one.
Keeping the record of prior computations is the key feature of dynamic programming that helps avoid recomputing known results.
Inspecting each of the $n^2$ node pairs in the graph $n$ times, one for each increase in path length, keeps the performance bound at O(n^3).
Because it finds the shortest path between all pairs, we end up with a complete set of paths across the graph at the end of the compution.
That is, we now have the full transitive closure in O(n^3) operations.
This is a clear improvement over using fast matrix multiplication for transitive closure.

What if we just want to compose a limited number of relations, like we did with the friends of friends.
We know even the fastest matrix multiply method will still require operations in the order of $O(n^{2.373})$.

Is this the best we can do?

Let's revisit the earlier observation that we are working with logical matrices.
Is it possible to leverage Boolean matrix multiplication as a performance advantage?

We can rewrite our left-side matrix of the multiplication as set of rows that just records the postion of the ones in that row.
We then step through each element of these these $i$ rows and inspect the corresponding element for its position $k$ in the right side matrix in cell $k,j$.
If the we find a $1$ in that position then we know there is a path between these elements.
We found one path so we don't need to do any more work. 
We set the value of the entry in the solution matrix to $1$.
We can then move on to the next row.
If we don't find a $1$ we keep going until we run out of entries in the second matrix to inspect.
If no ones occured, then the result matrix entry is set to $0$.
There are no paths between these elements.
The expected run time of this algorithms for most matrices is $O(n^2)$.
The are worst case peformance of $O(n^3)$ but that's not any worse than the solutions above.

This gives us an $O(n^2)$ algorithm for computing paths between nodes.
This means we could compute the friend of a friend relation for a one-million person social network in same time it would take to compute the relation for a 10,000 person network had we used naive matrix multiplication.
That's a huge increase in the size of networks we can explore.

There's a lot of value in thinking about perfomance when working with computers.